## 1. Setup and Data Preparation

We begin by importing the necessary packages and preparing the Brent oil price data. The log return series is used instead of raw prices, as it tends to be stationary and better captures structural shifts.

In [6]:
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns

# Load cleaned dataset
df = pd.read_csv("../data/brent_prices_cleaned.csv", parse_dates=["Date"], index_col="Date")
df['log_return'] = np.log(df['Price'] / df['Price'].shift(1))
df = df.dropna()
returns = df['log_return'].values
n = len(returns)


## 2. Bayesian Change Point Model in PyMC3

We implement a basic Bayesian model with a single change point in the mean. The variance is assumed constant for simplicity. The change point (`tau`) is modeled as a discrete uniform prior, while the means before and after (`mu_1`, `mu_2`) are drawn from normal priors.

In [ ]:
with pm.Model() as model:
    # Priors
    tau = pm.DiscreteUniform('tau', lower=0, upper=n)

    mu_1 = pm.Normal('mu_1', mu=0, sigma=1)
    mu_2 = pm.Normal('mu_2', mu=0, sigma=1)
    sigma = pm.HalfNormal('sigma', sigma=1)

    # Switch mean based on time index
    mu = pm.math.switch(tau >= np.arange(n), mu_1, mu_2)

    # Likelihood
    obs = pm.Normal('obs', mu=mu, sigma=sigma, observed=returns)

    # MCMC Sampling
    trace = pm.sample(2000, tune=1000, target_accept=0.95, return_inferencedata=True)


Multiprocess sampling (2 chains in 2 jobs)
CompoundStep
>Metropolis: [tau]
>NUTS: [mu_1, mu_2, sigma]


Output()

## 3. Posterior Diagnostics

We check convergence using ArviZ summaries and trace plots. The `r_hat` value should be close to 1.0, and the posterior of the change point (`tau`) should be tightly concentrated if a shift exists.

In [ ]:
az.summary(trace, var_names=["tau", "mu_1", "mu_2", "sigma"])
az.plot_trace(trace, var_names=["tau", "mu_1", "mu_2"])

## 4. Change Point Estimation

We extract the most probable index of the change point (`tau`) and convert it into a calendar date.

In [ ]:
# Most probable change point
tau_samples = trace.posterior['tau'].values.flatten()
tau_mean_idx = int(np.round(np.mean(tau_samples)))
change_date = df.index[tau_mean_idx]

print(f"Estimated change point date: {change_date.date()}")

## 5. Quantifying Impact

We calculate the posterior means before and after the detected change point to understand the shift in return behavior.

In [ ]:
mu_1_mean = trace.posterior['mu_1'].mean().item()
mu_2_mean = trace.posterior['mu_2'].mean().item()

print(f"Mean return before change: {mu_1_mean:.4f}")
print(f"Mean return after change:  {mu_2_mean:.4f}")
print(f"Change magnitude: {((mu_2_mean - mu_1_mean) / abs(mu_1_mean)) * 100:.2f}%")

## 6. Associating Change with Historical Event

We compare the detected change date with the timeline of major geopolitical or economic events to identify the most plausible cause.

In [ ]:
events = pd.read_csv("data/key_events.csv", parse_dates=["start_date"])
events['days_diff'] = abs(events['start_date'] - change_date)
events.sort_values("days_diff").head(3)

## 7. Visualizing the Change Point

To support interpretation, we overlay the change point on the log returns time series.

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(df['log_return'], label='Log Returns')
plt.axvline(change_date, color='red', linestyle='--', label='Detected Change Point')
plt.title('Log Returns with Detected Change Point')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('reports/img/change_point_detected.png')
plt.show()
